In [ ]:
import yt_dlp
import re
import pandas as pd
import os
from typing import List, Dict, Any, Set, Tuple

# 引入 pykakasi 進行日文轉換 (雖然流程簡化，但保留庫的導入)
try:
    from pykakasi import kakasi
    KKS = kakasi()
    KKS.setMode("J", "a")
    KKS.setMode("H", "a")
    KKS.setMode("K", "a")
    CONV = KKS.getConverter()
except ImportError:
    KKS = None
    pass


# ----------------- 檔案快取設定 -----------------
# **請確認此路徑是否正確**
BASE_DIR = r"C:\Users\Casper\Desktop\爬蟲\歌單" 
SETLIST_CACHE_PATH = os.path.join(BASE_DIR, "setlist_cache.txt")
VTUBER_LIST_PATH = os.path.join(BASE_DIR, "vtuber_list.txt") 

# ----------------- 輔助函式 (時間, 文本, ID 轉換, 相似度) -----------------

def time_to_seconds(t: str) -> int:
    """將時間字串 (MM:SS 或 HH:MM:SS) 轉換為總秒數。"""
    parts = [int(p) for p in t.split(":")]
    if len(parts) == 3:
        return parts[0]*3600 + parts[1]*60 + parts[2]
    elif len(parts) == 2:
        return parts[0]*60 + parts[1]
    return 0

def make_timestamp_url(video_url: str, seconds: int) -> str:
    """建立帶有時間戳的 YouTube URL。"""
    if "?" in video_url:
        return f"{video_url}&t={seconds}s"
    else:
        return f"{video_url}?t={seconds}s"

def clean_text(s: str) -> str:
    """清理並簡化文本，用於比對。"""
    s = str(s).lower()
    s = re.sub(r"\s+", "", s)
    # 保留中文、日文、英文、數字
    s = re.sub(r"[^\w\u4e00-\u9fff\u3040-\u30ff]", "", s)
    return s

def is_song_similar(song1: str, song2: str) -> bool:
    """判斷兩首歌名是否相似 (用於 30 秒去重邏輯)。"""
    c1 = clean_text(song1)
    c2 = clean_text(song2)
    
    # 至少有一邊為空，則不相似
    if not c1 or not c2:
        return False
    
    # **歌名相似度判斷邏輯：較短的清理後的歌名是否包含於較長的歌名中。**
    if len(c1) > len(c2):
        return c2 in c1
    else:
        return c1 in c2

def get_channel_id(streams_page_url: str) -> str:
    """從 /streams 網址中提取頻道 ID (例如 @vtubername)。"""
    match = re.search(r"youtube\.com/(?:(@[^/]+)|channel/([^/]+))", streams_page_url)
    if match:
        if match.group(1):
            return match.group(1).replace("@", "")
        if match.group(2):
            return match.group(2)
    return ""

def get_video_id(video_url: str) -> str:
    """從完整的 YouTube URL 提取唯一的影片 ID。"""
    match = re.search(r"(?:v=|youtu\.be/|/embed/)([0-9A-Za-z_-]{11})", video_url)
    if match:
        return match.group(1)
    return "" 

def get_channel_info_by_url(url: str) -> Tuple[str | None, str | None]:
    """使用 yt-dlp 獲取頻道名稱和頻道 ID。"""
    opts = {"quiet": True, "skip_download": True, "extract_flat": True, "geturl": True}
    try:
        with yt_dlp.YoutubeDL(opts) as ydl:
            info = ydl.extract_info(url, download=False)
            
            channel_id = info.get("uploader_id") or info.get("channel_id")
            channel_name = info.get("uploader") or info.get("channel")
            
            return channel_id, channel_name
            
    except Exception as e:
        print(f"⚠️ 獲取頻道資訊失敗: {e}")
        return None, None
        
# ----------------- **專用數據清理工具 (已啟用 30 秒 + 歌名邏輯)** -----------------
    
def clean_duplicate_cache():
    """
    對 setlist_cache.txt 執行 30 秒 + 歌名相似度去重。
    針對同一影片，如果多筆數據時間相差小於 30 秒且歌名相似，則只保留最早的一筆。
    """
    if not os.path.exists(SETLIST_CACHE_PATH):
        print("✅ 快取檔案不存在，無需清理。")
        return
        
    print(f"🧹 正在對現有快取檔案 ({SETLIST_CACHE_PATH}) 執行 **30 秒 + 歌名去重**...")
    
    # 結構: {video_id: [ (seconds, song_name, full_cache_line), ... ]}
    video_data: Dict[str, List[Tuple[int, str, str]]] = {}
    all_lines = []
    total_lines = 0
    
    try:
        with open(SETLIST_CACHE_PATH, "r", encoding="utf-8") as f:
            all_lines = f.readlines()
            total_lines = len(all_lines)
            
    except Exception as e:
        print(f"❌ 讀取快取檔案失敗: {e}")
        return

    # 1. 解析並按影片 ID 分組 (新增 song_name 欄位)
    for line in all_lines:
        try:
            parts = [p.strip() for p in line.strip().split(" | ")]
            if len(parts) != 7:
                continue

            video_id = parts[2]
            seconds = int(parts[4])
            song_name = parts[6] # 新增：原始歌曲名稱
            
            if video_id not in video_data:
                video_data[video_id] = []
            
            video_data[video_id].append((seconds, song_name, line.strip()))
            
        except Exception:
            continue

    final_unique_lines = set()
    
    # 2. 對每個影片執行 30 秒 + 歌名過濾
    for _, entries in video_data.items():
        # 按時間戳排序，確保先處理時間較早的項目
        entries.sort(key=lambda x: x[0]) 
        
        # 結構: [(seconds, song_name)] 儲存已保留的歌曲信息
        kept_songs: List[Tuple[int, str]] = []
        
        for seconds, song_name, line in entries:
            is_duplicate = False
            
            # 與所有已保留的歌曲進行比對
            for prev_second, prev_song_name in kept_songs:
                
                # 條件 1: 時間接近 (30 秒內)
                time_close = abs(seconds - prev_second) <= 30
                
                # 條件 2: 歌名相似
                name_similar = is_song_similar(song_name, prev_song_name)
                
                if time_close and name_similar:
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                # 通過檢查，保留這筆數據
                kept_songs.append((seconds, song_name))
                final_unique_lines.add(line) 

    # 3. 重新寫入檔案
    if total_lines > 0:
        with open(SETLIST_CACHE_PATH, "w", encoding="utf-8") as f:
            for line in sorted(list(final_unique_lines)):
                f.write(line + "\n")
                
        deleted_count = total_lines - len(final_unique_lines)
        if deleted_count > 0:
            print(f"✅ 清理完成！已移除 {deleted_count} 筆重複或時間相近/歌名相似的數據。")
        else:
            print("✅ 無發現需要清理的重複數據。")
    else:
        print("✅ 快取檔案為空，無需清理。")

# ----------------- 歌枠影片抓取與解析 -----------------
KEYWORDS = ["歌枠", "歌ってみた", "Singing"]

def get_singing_stream_urls(streams_page_url: str) -> List[Dict[str, str]]:
    """從 /streams 頁面抓取歌枠影片的 URL 和標題。"""
    opts = {
        "quiet": True,
        "skip_download": True,
        "extract_flat": True
    }
    try:
        with yt_dlp.YoutubeDL(opts) as ydl:
            info = ydl.extract_info(streams_page_url, download=False)
    except Exception as e:
        print(f"抓取影片清單失敗: {e}")
        return []
    results = []
    for item in info.get("entries", []):
        title = item.get("title") or ""
        if any(k.lower() in title.lower() for k in KEYWORDS):
            vid = item.get("id") or item.get("url")
            if not vid:
                continue
            if not vid.startswith("http"):
                vid = f"https://www.youtube.com/watch?v={vid}"
            results.append({"title": title, "url": vid})
    return results

SETLIST_KEYWORDS = [
    "セトリ", "SET LIST", "Setlist", "set list",
    "タイムスタンプ", "Time Stamp", "timestamp", "Timestamps",
    "曲順", "Song List", "Playlist","せとりんりん","Set List","セットリスト"
]
TIME_PATTERN = r"(\d{1,2}:\d{2}(?::\d{2})?)"
TIME_RANGE_PATTERN = r"(\d{1,2}:\d{2}(?::\d{2})?)\s*(?:～|-)\s*\d{1,2}:\d{2}(?::\d{2})?"

def extract_setlist_lines(text: str) -> pd.DataFrame | None:
    """從文本中解析出 Setlist 的歌曲和時間。"""
    lines = text.splitlines()
    data = []
    for line in lines:
        start_time = None
        match_range = re.search(TIME_RANGE_PATTERN, line)
        if match_range:
            start_time = match_range.group(1).strip()
            song = line.replace(match_range.group(0), "").strip()
        else:
            match = re.search(TIME_PATTERN, line)
            if match:
                start_time = match.group(1).strip()
                song = line.replace(start_time, "").strip()
            else:
                continue
        song = re.sub(r"[『』'\"～\-:：]+", "", song).strip()
        if not song:
            song = line.strip() 
        song = re.sub(TIME_PATTERN, "", song).strip()
        if song and start_time:
            data.append({"Song": song, "Start Time": start_time})
    if data:
        return pd.DataFrame(data)
    return None

def is_valid_setlist(text: str) -> bool:
    """判斷文本是否為有效的 Setlist。"""
    if any(k in text for k in SETLIST_KEYWORDS):
        return True
    df = extract_setlist_lines(text)
    if df is not None and not df.empty:
        return True
    return False

def get_clean_setlist_from_video(video_url: str) -> List[Dict[str, Any]]:
    """從影片評論中提取有效的 Setlist。"""
    opts = {
        "quiet": True,
        "skip_download": True,
        "getcomments": True,
        "comment_limit": 100 
    }
    try:
        with yt_dlp.YoutubeDL(opts) as ydl:
            info = ydl.extract_info(video_url, download=False)
        if info is None:
            return []
    except Exception:
        return []

    comments = info.get("comments") or []
    setlist_comments = []

    for c in comments:
        text = c.get("text") or c.get("textDisplay") or ""
        if is_valid_setlist(text):
            df = extract_setlist_lines(text)
            if df is not None:
                setlist_comments.append({
                    "author": c.get("author"),
                    "data": df
                })
    return setlist_comments

# ----------------- VTuber List 快取處理 -----------------

def load_vtuber_list() -> Dict[str, Dict[str, str]]:
    """讀取 vtuber_list.txt，返回 ID 到資訊的字典。"""
    vtubers = {}
    if not os.path.exists(VTUBER_LIST_PATH):
        return vtubers
        
    with open(VTUBER_LIST_PATH, "r", encoding="utf-8") as f:
        for line in f:
            try:
                # 格式: ID | Name | StreamsURL
                parts = [p.strip() for p in line.strip().split(" | ")]
                if len(parts) == 3:
                    vtubers[parts[0]] = {
                        "name": parts[1], 
                        "url": parts[2]
                    }
            except Exception:
                continue
    return vtubers

def save_vtuber_list(vtubers: Dict[str, Dict[str, str]]):
    """將 VTuber 列表寫入 vtuber_list.txt。"""
    os.makedirs(BASE_DIR, exist_ok=True)
    with open(VTUBER_LIST_PATH, "w", encoding="utf-8") as f:
        for vid, info in vtubers.items():
            f.write(f"{vid} | {info['name']} | {info['url']}\n")

def add_vtuber_to_list(channel_id: str, channel_name: str, streams_url: str):
    """新增 VTuber 到列表並儲存。"""
    vtubers = load_vtuber_list()
    vtubers[channel_id] = {
        "name": channel_name, 
        "url": streams_url
    }
    save_vtuber_list(vtubers)

# ----------------- Setlist 快取讀寫與搜尋 -----------------

def get_cached_video_ids(channel_id: str) -> Set[str]:
    """讀取快取，返回該頻道所有已快取的影片 ID 集合。"""
    cached_ids = set()
    if not os.path.exists(SETLIST_CACHE_PATH):
        return cached_ids

    with open(SETLIST_CACHE_PATH, "r", encoding="utf-8") as f:
        for line in f:
            try:
                parts = [p.strip() for p in line.strip().split(" | ")]
                if len(parts) == 7 and parts[0] == channel_id:
                    cached_ids.add(parts[2]) 
            except Exception:
                continue
    return cached_ids

def write_setlist_to_cache(video_url: str, video_title: str, channel_id: str, filtered_songs: List[Dict[str, str]]):
    """
    將經過 30 秒過濾後的 Setlist 數據寫入 setlist_cache.txt。
    
    :param filtered_songs: 格式為 [{"Song": "...", "Start Time": "..."}, ...] 的列表
    """
    os.makedirs(BASE_DIR, exist_ok=True)
    
    video_id = get_video_id(video_url)
    if not video_id:
        return

    # 確保本次 Setlist 寫入的行是唯一的 (針對不同評論可能產生相同數據)
    seen_lines = set() 
    
    try:
        with open(SETLIST_CACHE_PATH, "a", encoding="utf-8") as f:
            for song_data in filtered_songs:
                seconds = time_to_seconds(song_data["Start Time"])
                
                # 建立固定格式的單行字串 (7 個欄位)
                line_data = [
                    channel_id, 
                    video_url, 
                    video_id,  
                    video_title.replace(" | ", " - "), 
                    str(seconds), 
                    song_data['Start Time'], 
                    song_data['Song'].replace(" | ", " - ")
                ]
                line = " | ".join(line_data) + "\n"
                
                if line not in seen_lines:
                    f.write(line)
                    seen_lines.add(line)
                        
    except Exception as e:
        print(f"寫入 Setlist 快取失敗: {e}")


def update_channel_cache(channel_id: str, current_video_ids: Set[str]):
    """更新快取：移除已下架的影片資料。"""
    if not os.path.exists(SETLIST_CACHE_PATH):
        return
    
    all_lines = []
    lines_to_keep = []
    deleted_video_ids = set()
    
    try:
        with open(SETLIST_CACHE_PATH, "r", encoding="utf-8") as f:
            all_lines = f.readlines()
    except Exception as e:
        print(f"讀取快取檔案失敗: {e}")
        return

    for line in all_lines:
        try:
            parts = [p.strip() for p in line.strip().split(" | ")]
            if len(parts) != 7:
                lines_to_keep.append(line)
                continue

            cache_channel_id, _, video_id, _, _, _, _ = parts
            
            if cache_channel_id != channel_id:
                lines_to_keep.append(line)
            else:
                if video_id in current_video_ids:
                    lines_to_keep.append(line)
                else:
                    deleted_video_ids.add(video_id)
        except Exception:
            lines_to_keep.append(line)

    if deleted_video_ids:
        print(f"🧹 已移除 {len(deleted_video_ids)} 個已下架影片的相關快取資料。")
        os.makedirs(BASE_DIR, exist_ok=True)
        with open(SETLIST_CACHE_PATH, "w", encoding="utf-8") as f:
            f.writelines(lines_to_keep)


def search_from_cache(search_clean: str, channel_id: str) -> List[Tuple[str, str, str]]:
    """
    從 setlist_cache.txt 中搜尋指定的歌曲。
    返回格式: (Channel ID, URL, Song Name)
    """
    results = []
    
    if not os.path.exists(SETLIST_CACHE_PATH):
        return results

    with open(SETLIST_CACHE_PATH, "r", encoding="utf-8") as f:
        for line in f:
            try:
                parts = [p.strip() for p in line.strip().split(" | ")]
                if len(parts) != 7:
                    continue

                cache_channel_id = parts[0]
                video_url = parts[1]
                seconds_str = parts[4]
                song_name = parts[6]
                
                # 篩選邏輯：如果 channel_id 不為 '*' 則進行比對
                if channel_id != '*' and cache_channel_id != channel_id:
                    continue
                    
                if search_clean in clean_text(song_name):
                    seconds = int(seconds_str)
                    direct_url = make_timestamp_url(video_url, seconds)
                    
                    # 返回 Channel ID, 帶時間戳的 URL, 和原始歌曲名
                    results.append((cache_channel_id, direct_url, song_name))
            except Exception:
                continue

    # 進行去重 (基於 URL 進行去重，因為一首歌可能有多個 Setlist 評論)
    unique_results = {}
    for cid, url, song in results:
        if url not in unique_results:
             # 由於要保留歌曲名，我們用 Tuple 儲存
            unique_results[url] = (cid, url, song)

    # 按 Channel ID 排序，以便後續分組
    return sorted(list(unique_results.values()), key=lambda x: x[0])


# ----------------- 輸出與選擇邏輯 -----------------

def display_vtuber_selection(vtubers: Dict[str, Dict[str, str]]) -> Dict[str, str] | None:
    """顯示所有 VTuber 選項，並處理用戶輸入。"""
    
    print("\n--- 請選擇 VTuber 頻道 ---")
    
    # 將所有 VTuber 資訊轉換成列表並依名稱排序
    display_list = sorted([(v['name'], k, v['url']) for k, v in vtubers.items()], key=lambda x: x[0].lower())
    
    vtuber_map = {}
    
    # 1. 列出 VTuber
    for i, (name, vid, url) in enumerate(display_list):
        vtuber_map[str(i+1)] = {'id': vid, 'url': url}
        # 顯示格式：[編號]. 頻道名稱 (ID)
        print(f"{i+1}. **{name}**")

    # 2. 加入特殊選項
    total_options = len(display_list)
    
    # 貼新資料的直播頁面網址
    vtuber_map[str(total_options + 1)] = {'id': 'NEW', 'url': None}
    print(f"{total_options + 1}. **貼新資料的直播頁面網址**")

    # 不限 vtuber
    vtuber_map[str(total_options + 2)] = {'id': 'ALL_VTUBERS', 'url': None}
    print(f"{total_options + 2}. **不限 vtuber** (搜尋所有快取資料)")
    
    print("-" * 20)

    while True:
        choice = input("請輸入數字選擇頻道：> ").strip()
        if choice in vtuber_map:
            return vtuber_map[choice]
        print("輸入無效，請重新輸入。")


def format_and_display_results(search_results: List[Tuple[str, str, str]], vtuber_list: Dict[str, Dict[str, str]]):
    """
    格式化並分組輸出搜尋結果。
    
    :param search_results: 格式為 (Channel ID, URL, Song Name) 的列表
    :param vtuber_list: Channel ID 到資訊 (Name, URL) 的字典
    """
    
    # 按 Channel ID 進行分組
    grouped_results: Dict[str, List[Tuple[str, str]]] = {} # {Channel ID: [(URL, Song Name), ...]}
    for cid, url, song in search_results:
        if cid not in grouped_results:
            grouped_results[cid] = []
        grouped_results[cid].append((url, song))

    total_count = len(search_results)
    
    print("-" * 40)
    if total_count == 0:
        print("😢 在 Setlist 快取中找不到這首歌。")
        return

    print(f"🎉 總共找到 {total_count} 筆歌曲網址，分佈於 {len(grouped_results)} 個頻道：")
    
    # 輸出每個分組
    for cid, items in grouped_results.items():
        # 獲取頻道名稱，若找不到則使用 ID
        channel_name = vtuber_list.get(cid, {}).get('name', f"未知頻道 ({cid})")
        
        print(f"\n## 🎤 {channel_name} ({len(items)} 筆)")
        
        # 輸出該頻道下的每個結果
        for url, song_name in items:
            print(f"* 歌曲名稱: **{song_name}**")
            print(f"  連結: {url}")
            
    print("-" * 40)


# ----------------- **主程式運行入口** -----------------

if __name__ == "__main__":
    
    # **第一步：清理現有快取 (執行 30 秒 + 歌名相似度邏輯去重)**
    clean_duplicate_cache() 
    
    search_song = input("請輸入要找的歌名：> ").strip()
    if not search_song:
        print("歌名不能為空！")
        exit()

    search_clean = clean_text(search_song)
    final_results = []
    vtuber_list = load_vtuber_list()
    
    is_all_vtubers_search = False

    # 決定頻道來源
    if not vtuber_list:
        print("\n--- VTuber 列表為空 ---")
        streams_url = input("請直接貼新資料的直播頁面網址：> ").strip()
        if not streams_url:
            print("網址不能為空！")
            exit()
        selected_info = {'id': 'NEW', 'url': streams_url}
    else:
        selected_info = display_vtuber_selection(vtuber_list)
        
    print("-" * 40)
    
    # 處理選擇結果
    
    if selected_info['id'] == 'ALL_VTUBERS':
        # 情況一: 不限 VTuber (搜尋所有快取)
        is_all_vtubers_search = True
        print(f"🔍 正在從所有快取資料中搜尋 '{search_song}'...")
        final_results = search_from_cache(search_clean, channel_id='*') 
        
    else:
        # 情況二: 處理單一 VTuber 或新增 VTuber
        
        streams_url_to_process = None
        
        if selected_info['id'] == 'NEW':
            streams_url_to_process = input("請貼上新的直播頁面網址：> ").strip()
            if not streams_url_to_process:
                print("網址不能為空！")
                exit()
        else:
            streams_url_to_process = selected_info['url']
            
        channel_id, channel_name = get_channel_info_by_url(streams_url_to_process)
        
        if not channel_id or not channel_name:
             print("❌ 無法從網址中獲取頻道 ID 和名稱，請確認網址格式。")
             exit()
             
        if channel_id not in vtuber_list:
            add_vtuber_to_list(channel_id, channel_name, streams_url_to_process)
            print(f"✅ 已將新的 VTuber 頻道 **{channel_name}** 加入列表。")

        # 標準的資料抓取/維護/搜尋流程
        
        print(f"\n📡 正在處理頻道 **{channel_name}** (@{channel_id})...")
        current_videos = get_singing_stream_urls(streams_url_to_process)
        current_video_ids = {get_video_id(v["url"]) for v in current_videos}
        
        if not current_videos:
            print("⚠️ 完全沒抓到歌枠。")
            if os.path.exists(SETLIST_CACHE_PATH):
                update_channel_cache(channel_id, set()) 
            final_results = search_from_cache(search_clean, channel_id) 
            
        else:
            # 移除已下架影片的快取
            update_channel_cache(channel_id, current_video_ids)

            cached_video_ids = get_cached_video_ids(channel_id)
            videos_to_process = [
                v for v in current_videos 
                if get_video_id(v["url"]) not in cached_video_ids
            ]
            
            if videos_to_process:
                print(f"🔄 發現 {len(videos_to_process)} 個新影片，開始抓取 Setlist 並快取...")
                
                for idx, v in enumerate(videos_to_process):
                    url = v["url"]
                    title = v["title"]
                    print(f"處理進度: {idx+1}/{len(videos_to_process)} - {title[:30]}...")
                    
                    try:
                        setlists = get_clean_setlist_from_video(url)
                    except Exception:
                        continue
                    
                    if not setlists:
                        continue
                    
                    filtered_songs_to_cache = []
                    # 結構: [(seconds, song_name)] 儲存已保留的歌曲信息
                    kept_songs: List[Tuple[int, str]] = [] 
                    
                    # **執行新數據的 30 秒 + 歌名比對邏輯**
                    for s in setlists:
                        df = s["data"]
                        for _, row in df.iterrows():
                            current_song_name = row["Song"]
                            seconds = time_to_seconds(row["Start Time"])
                            
                            is_duplicate = False
                            
                            # 與所有已保留的歌曲進行比對
                            for prev_second, prev_song_name in kept_songs:
                                
                                # 條件 1: 時間接近 (30 秒內)
                                time_close = abs(seconds - prev_second) <= 30
                                
                                # 條件 2: 歌名相似
                                name_similar = is_song_similar(current_song_name, prev_song_name)
                                
                                if time_close and name_similar:
                                    # 發現重複，跳出比對
                                    is_duplicate = True
                                    break
                            
                            if not is_duplicate:
                                # 通過檢查，記錄並加入待寫入列表
                                kept_songs.append((seconds, current_song_name))
                                filtered_songs_to_cache.append({
                                    "Song": current_song_name,
                                    "Start Time": row["Start Time"]
                                })
                    
                    if filtered_songs_to_cache:
                        # 寫入快取
                        write_setlist_to_cache(url, title, channel_id, filtered_songs_to_cache)

            else:
                print("✅ 所有影片皆已快取，無需新增。")
                
            # 從快取中進行最終搜尋
            print(f"🔍 正在從最新的快取中搜尋 '{search_song}'...")
            final_results = search_from_cache(search_clean, channel_id)

    # 5. 最終結果輸出
    format_and_display_results(final_results, vtuber_list)